In [148]:
import os
from google.cloud.sql.connector import Connector, IPTypes
import sqlalchemy
from sqlalchemy import MetaData, Table, create_engine
import os
from dotenv import load_dotenv
import pandas as pd

import json
import requests

load_dotenv()

connector = Connector()

def getconn():
    with Connector() as connector:
        conn = connector.connect(
            "surfmetrics-332511:europe-west9:prod-presse",
            "pg8000",
            user="admin",
            password="c0[\V;7u(ym=)*k,",
            db="surfmetrics",
            ip_type= IPTypes.PUBLIC
        )
    return conn


# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine("postgresql+pg8000://",creator=getconn)


relation_table_name = '"company_X_articles"'
relation_table_name_2 = '"portfolios_X_companies"  '
sql_string = f"SELECT *   \
                            FROM {relation_table_name}    \
                            WHERE COMPANY_ID IN \
                                    (SELECT COMPANY_ID  \
                                        FROM {relation_table_name_2} \
                                        WHERE PORTFOLIO_ID = 1 )    \
                                AND PROCESSING_ERROR = FALSE    \
                                AND TOTAL_MENTIONS > 0     \
                                AND SCORE BETWEEN 1 AND 100 "




with pool.connect() as db_conn:
    select_query = sqlalchemy.text(sql_string)
    results = db_conn.execute(select_query).all()
    df_pa = pd.DataFrame(results)

db_conn.close()

In [149]:
sql_string = f"SELECT * from ARTICLES"

with pool.connect() as db_conn:
    select_query = sqlalchemy.text(sql_string)
    results = db_conn.execute(select_query).all()
    df_article = pd.DataFrame(results)

db_conn.close()

In [213]:
sql_string = f"SELECT * from topic"

with pool.connect() as db_conn:
    select_query = sqlalchemy.text(sql_string)
    results = db_conn.execute(select_query).all()
    df_topic = pd.DataFrame(results)

db_conn.close()

In [154]:
df_article.columns

Index(['url_hash', 'url', 'provider', 'title', 'html_content', 'summary',
       'keywords', 'language', 'category', 'pending', 'date_published',
       'date_published_zen', 'date_created', 'processed_date',
       'processed_date_scraping', 'processed_date_filter', 'google_search',
       'google_news', 'date_visual', 'subject'],
      dtype='object')

In [155]:
join_table = pd.merge(df_pa, df_article[['url_hash', 'category','date_published']], left_on='article_url_hash', right_on='url_hash', how='left')


In [161]:
join_fin = join_table[join_table['category']!='Other'][['article_url_hash','score','category','date_published']]

In [205]:
df_topic = df_topic.sample(len(df_topic)).reset_index(drop=True)

In [207]:
fin_df = pd.DataFrame(columns=['id', 'article_url_hash'])
start = 7
end = 100 - (100 - start) % 4 # adjust end value to ensure last check is until 100
for id in df_topic['id']:
    sample = join_fin[join_fin['score'].between(start, start+3)]
    new = df_topic[df_topic['id']==id]['id'].to_frame().merge(sample['article_url_hash'].to_frame() , how='cross').reset_index(drop=True)
    fin_df = pd.concat([fin_df,new ], axis = 0).reset_index(drop=True)
    start+=4
    if start > end:
        break
print(end)


99


In [208]:
fin_df['priority'] = 1

In [209]:
fin_df.columns

Index(['id', 'article_url_hash', 'priority'], dtype='object')

In [210]:
fin_df.columns = ["topic_id","article_url_hash","priority"]

In [212]:
fin_df.to_csv('topic_x_article.csv', sep=',', index=False)